## fiducial marker pattern summary

This notebook calculates summary information for different aerial camera fiducial marker patterns, based on browse images and calibration reports downloaded from [EarthExplorer](https://earthexplorer.usgs.gov/). The summary information includes:

- fiducial marker separation distance (mean ± standard deviation, median)
- where available, fiducial marker location: x, y position (mean ± standard deviation) and angle with respect to the principal point coordinate system (in degrees)
- number of reports, summarized by camera names

For examples of what each pattern looks like, see https://spymicmac.readthedocs.io/en/cameras.html.

In [ ]:
import pandas as pd
from scripts import tools


cal_info = pd.read_csv('data/calibration_information_cleaned.csv')
cal_info['camera_make'] = cal_info['camera_make'].fillna('').astype(str)
cal_info['camera_model'] = cal_info['camera_model'].fillna('').astype(str)
cal_info['camera_name'] = cal_info[['camera_make', 'camera_model']].agg(' '.join, axis=1).str.strip()

cal_info = cal_info.drop_duplicates(subset=['cal_file', 'camera_name', 'marker_style'])

In [ ]:
print("Number of reports for each marker pattern:")
print(cal_info.value_counts('marker_style')) # print a summary of the number of reports for each marker style

## wing-type fiducials

Print statistics tables for the following pattern types:

- hollow_wing
- small_flat
- large_flat
- tall_wing
- curved_flat
- round_wing
- sharp_wing

Also print the count of camera names corresponding to each pattern.

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'hollow_wing'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'hollow_wing'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'small_flat'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'small_flat'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'large_flat'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'large_flat'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'tall_wing'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'tall_wing'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'curved_flat'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'curved_flat'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'curved_wing'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'curved_wing'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'sharp_wing'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'sharp_wing'].value_counts('camera_name'))

## fairchild

Print statistics tables for the following pattern types:

- midside_notch
- corner_notch
- t5_notch

Also print the count of camera names corresponding to each pattern.

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'midside_notch'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'midside_notch'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'corner_notch'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'corner_notch'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 't5_notch'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 't5_notch'].value_counts('camera_name'))

## park

Print statistics tables for 'park_round' pattern.

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'park_round'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'park_round'].value_counts('camera_name'))

## wild

Print statistics tables for the following pattern types:

- light_corner
- wild_corner
- wild_all_cross
- wild_mid_crosshair
- wild_all_crosshair

Also print the count of camera names corresponding to each pattern.

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'light_corner'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'light_corner'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'wild_corner'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'wild_corner'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'wild_all_cross'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'wild_all_cross'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'wild_mid_crosshair'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'wild_mid_crosshair'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'wild_all_crosshair'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'wild_all_crosshair'].value_counts('camera_name'))

## zeiss rmk

Print statistics tables for the following pattern types:

- zeiss_mid
- zeiss_fixed
- zeiss_float

Also print the count of camera names corresponding to each pattern.

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'zeiss_mid'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'zeiss_mid'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'zeiss_float'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'zeiss_float'].value_counts('camera_name'))

In [ ]:
tools.nice_table(cal_info.loc[cal_info.marker_style == 'zeiss_fixed'])

print("\nNumber of reports for each camera make/model:")
print(cal_info.loc[cal_info.marker_style == 'zeiss_fixed'].value_counts('camera_name'))